In [1]:
import numpy as np
import pandas as pd

In [2]:
tr = pd.read_csv('train_data.csv')

In [3]:
fe = pd.read_csv('farm_data-1646897931981.csv')

In [4]:
#fe.shape

In [5]:
fe.drop_duplicates(subset=['farm_id'], inplace=True)

In [6]:
#fe.shape

In [7]:
tr.shape

(20216100, 4)

In [8]:
trfe = pd.merge(tr, fe, how="left", on=["farm_id"])

In [9]:
#trfe.shape

In [10]:
tw = pd.read_csv('train_weather-1646897968670.csv')

In [11]:
#tw.shape

In [12]:
trfe.rename(columns = {'date':'timestamp'},inplace = True)

In [13]:
train = pd.merge(trfe, tw, how="left", on=["deidentified_location", "timestamp"])

In [14]:
#train.head()

In [15]:
train.shape

(20216100, 16)

In [16]:
#train.isnull().sum()

In [17]:
#train_c = train.copy()

In [18]:
#train.info()

In [19]:
train['Unix Sec'] = pd.to_datetime(train['timestamp']).astype(int)/ 10**9

/var/folders/hk/d9j983wj2t35c0rlbmr4ldjw0000gn/T/ipykernel_11154/3716296932.py:1: FutureWarning: casting datetime64[ns] values to int64 with .astype(...) is deprecated and will raise in a future version. Use .view(...) instead.
  train['Unix Sec'] = pd.to_datetime(train['timestamp']).astype(int)/ 10**9


In [20]:
train['Unix Sec'] = train['Unix Sec'].astype('int64')

In [21]:
#train.head()

In [22]:
train.drop(['timestamp', 'operations_commencing_year', 'num_processing_plants', 'farm_id'], axis=1, inplace=True)

In [23]:
#train.isnull().sum()

In [24]:
"""import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline"""

'import seaborn as sns\nimport matplotlib.pyplot as plt\n%matplotlib inline'

In [25]:
"""corrMatrix = train.corr(method = 'pearson',min_periods = 1)"""

"corrMatrix = train.corr(method = 'pearson',min_periods = 1)"

In [26]:
"""plt.figure(figsize=(15,6))
sns.heatmap(corrMatrix, annot=True)
plt.show()"""

'plt.figure(figsize=(15,6))\nsns.heatmap(corrMatrix, annot=True)\nplt.show()'

## Missing value impute

In [27]:
missing_cols = [ "temp_obs", "cloudiness", "wind_direction", "dew_temp", "pressure_sea_level", "precipitation", "wind_speed"]

In [28]:
from sklearn.impute import SimpleImputer
imputer = SimpleImputer(strategy='median')

In [29]:
for col in missing_cols:
    imputer.fit(train[col].values.reshape(-1, 1))
    train[col] = imputer.transform(train[col].values.reshape(-1, 1))

In [30]:
train.isnull().sum()

ingredient_type          0
yield                    0
farm_area                0
farming_company          0
deidentified_location    0
temp_obs                 0
cloudiness               0
wind_direction           0
dew_temp                 0
pressure_sea_level       0
precipitation            0
wind_speed               0
Unix Sec                 0
dtype: int64

## Label Encoding

In [31]:
from sklearn.preprocessing import LabelEncoder

In [32]:
cat_cols = ["ingredient_type", "farming_company", "deidentified_location"]
le = LabelEncoder()
for col in cat_cols:
    train[col] = le.fit_transform(train[col])

In [33]:
#train.head()

In [34]:
#train.dtypes

## Scaling

In [35]:
#train.head()

In [36]:
from sklearn.preprocessing import StandardScaler

In [37]:
sc = StandardScaler()
train = pd.DataFrame(sc.fit_transform(train), columns=train.columns)
#train = sc.fit_transform(train)

## Model Building

In [38]:
#train.insert(0, 'id', train.index)

In [39]:
from sklearn.model_selection import train_test_split

In [40]:
"""y = train['yield']
X = train.loc[:, train.columns != 'yield']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state = 11)

print(X_train.shape, X_test.shape)
print(y_train.shape, y_test.shape)"""

"y = train['yield']\nX = train.loc[:, train.columns != 'yield']\n\nX_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state = 11)\n\nprint(X_train.shape, X_test.shape)\nprint(y_train.shape, y_test.shape)"

In [41]:
y = train['yield']
X = train.loc[:, train.columns != 'yield']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state = 11)

# Keep only 50% of the training set
X_train = X_train.sample(frac=0.2, random_state=11)
y_train = y_train[X_train.index]

X_test = X_test.sample(frac=0.2, random_state=11)
y_test = y_test[X_test.index]

print(X_train.shape, X_test.shape)
print(y_train.shape, y_test.shape)

(2830254, 12) (1212966, 12)
(2830254,) (1212966,)


In [42]:
#train.head()

In [43]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

In [44]:
#train.shape

In [45]:
rf = RandomForestRegressor()

In [46]:
rf.fit(X_train, y_train)

RandomForestRegressor()

In [47]:
#import pickle

In [48]:
"""import pickle
  
# Save the trained model as a pickle string.
saved_model = pickle.dumps(rf)
  
# Load the pickled model
#rf_from_pickle = pickle.loads(saved_model)
  
# Use the loaded pickled model to make predictions
#rf_from_pickle.predict(X_test)"""

'import pickle\n  \n# Save the trained model as a pickle string.\nsaved_model = pickle.dumps(rf)\n  \n# Load the pickled model\n#rf_from_pickle = pickle.loads(saved_model)\n  \n# Use the loaded pickled model to make predictions\n#rf_from_pickle.predict(X_test)'

In [49]:
"""with open('model.pkl', 'wb') as file:
    pickle.dump(model, file)"""

"with open('model.pkl', 'wb') as file:\n    pickle.dump(model, file)"

In [50]:
"""with open('model.pkl', 'rb') as file:
    loaded_model = pickle.load(file)

SyntaxError: EOF while scanning triple-quoted string literal (559434926.py, line 2)

In [ ]:
#prediction = loaded_model.predict(X)

### Prediction

In [51]:
train_pred1 = rf.predict(X_train)
test_pred1 = rf.predict(X_test)

In [52]:
train_pred1

array([-0.01281505, -0.01248104, -0.01167341, ..., -0.01313318,
       -0.01367165, -0.00791007])

In [53]:
test_pred1

array([-0.01114749, -0.00521065, -0.01378913, ..., -0.0013333 ,
       -0.01279086, -0.0136999 ])

In [ ]:
print(rf.score(X_train, y_train))
print(rf.score(X_test, y_test))

### RMSE

In [54]:
def convert_for_sklearn(label_list):
    return [1 if i == 1 else 0 for i in label_list]


def rmse_metric(y_true, y_pred):
    
    y_test_scoring = convert_for_sklearn(y_true)
    test_pred_scoring = convert_for_sklearn(y_pred)

    mse = mean_squared_error(y_true= y_test, y_pred = test_pred1)
    rmse = np.sqrt(mse)
    
    print("Test RMSE_score: ",rmse)

In [55]:
rmse_metric(y_true = y_test, y_pred = test_pred1)

Test RMSE_score:  0.2795875102225083


### Processing the test dataset

In [56]:
test_data = pd.read_csv('test_data-1664552867678.csv')

In [57]:
test_data.shape

(20848800, 4)

In [ ]:
test_data.head()

In [ ]:
fe.shape

In [58]:
tefe = pd.merge(test_data, fe, how="left", on=["farm_id"])

In [ ]:
tefe.shape

In [59]:
tew = pd.read_csv('test_weather-1646897984996-1664552604982.csv')

In [60]:
tefe.rename(columns = {'date':'timestamp'},inplace = True)

In [61]:
test = pd.merge(tefe, tew, how="left", on=["deidentified_location", "timestamp"])

In [62]:
test.shape

(20848800, 16)

In [63]:
test['Unix Sec'] = pd.to_datetime(test['timestamp']).astype(int)/ 10**9
test['Unix Sec'] = test['Unix Sec'].astype('int64')

/var/folders/hk/d9j983wj2t35c0rlbmr4ldjw0000gn/T/ipykernel_11154/2964248569.py:1: FutureWarning: casting datetime64[ns] values to int64 with .astype(...) is deprecated and will raise in a future version. Use .view(...) instead.
  test['Unix Sec'] = pd.to_datetime(test['timestamp']).astype(int)/ 10**9


In [64]:
test.drop(['timestamp', 'operations_commencing_year', 'num_processing_plants', 'farm_id'], axis=1, inplace=True)

In [65]:
missing_cols = [ "temp_obs", "cloudiness", "wind_direction", "dew_temp", "pressure_sea_level", "precipitation", "wind_speed"]

In [66]:
from sklearn.impute import SimpleImputer
imputer = SimpleImputer(strategy='median')

In [67]:
for col in missing_cols:
    imputer.fit(test[col].values.reshape(-1, 1))
    test[col] = imputer.transform(test[col].values.reshape(-1, 1))

In [68]:
test.isnull().sum()

ingredient_type          0
id                       0
farm_area                0
farming_company          0
deidentified_location    0
temp_obs                 0
cloudiness               0
wind_direction           0
dew_temp                 0
pressure_sea_level       0
precipitation            0
wind_speed               0
Unix Sec                 0
dtype: int64

In [69]:
from sklearn.preprocessing import LabelEncoder

In [70]:
cat_cols = ["ingredient_type", "farming_company", "deidentified_location"]
le = LabelEncoder()
for col in cat_cols:
    test[col] = le.fit_transform(test[col])

In [ ]:
test.head()

In [71]:
test.drop('id', axis=1, inplace=True)
#t = test.drop('id', axis=1)

In [72]:
sc = StandardScaler()
test = pd.DataFrame(sc.fit_transform(test), columns=test.columns)

In [ ]:
#test.insert(0, 'id', test.index)

In [ ]:
test.shape

In [73]:
batches = np.array_split(test, 10)

### Prediction for test

In [74]:
all_predictions = []
for i, batch in enumerate(batches):
    test_pred = rf.predict(batch)
    all_predictions.append(pd.Series(test_pred))
    print(f"Processed batch {i+1} of {len(batches)}")
    
predictions_df = pd.concat(all_predictions)

Processed batch 1 of 10
Processed batch 2 of 10
Processed batch 3 of 10
Processed batch 4 of 10
Processed batch 5 of 10
Processed batch 6 of 10
Processed batch 7 of 10
Processed batch 8 of 10
Processed batch 9 of 10
Processed batch 10 of 10


In [ ]:
#test_pred = rf.predict(Final_data_test)

In [75]:
predictions_df

0         -0.013816
1         -0.013816
2         -0.013816
3         -0.013816
4         -0.013816
             ...   
2084875   -0.013803
2084876   -0.013223
2084877   -0.011989
2084878   -0.012212
2084879   -0.013262
Length: 20848800, dtype: float64

In [76]:
type(predictions_df)

pandas.core.series.Series

In [77]:
predictions_df.shape

(20848800,)

### Submission file

In [78]:
test_data.id

0                  0
1                  1
2                  2
3                  3
4                  4
              ...   
20848795    41696845
20848796    41696846
20848797    41696847
20848798    41696848
20848799    41696849
Name: id, Length: 20848800, dtype: int64

In [79]:
iddf = pd.DataFrame({'id': test_data.id})

In [80]:
predf = pd.DataFrame({'yield': predictions_df})

In [81]:
iddf.shape

(20848800, 1)

In [82]:
predf.shape

(20848800, 1)

In [92]:
iddf = iddf.reset_index(drop=True)

In [94]:
predf = predf.reset_index(drop=True)

In [93]:
len(iddf.index.duplicated())

20848800

In [95]:
len(predf.index.duplicated())

20848800

In [96]:
submission1 = pd.concat([iddf, predf], axis=1, ignore_index=True)

In [ ]:
#submission = pd.DataFrame({'id': test_data.id, 'yield': predictions_df})

In [98]:
submission1

,0,1
0,0,-0.013816
1,1,-0.013816
2,2,-0.013816
3,3,-0.013816
4,4,-0.013816
...,...,...
20848795,41696845,-0.013803
20848796,41696846,-0.013223
20848797,41696847,-0.011989
20848798,41696848,-0.012212


In [99]:
submission1.to_csv('submission1.csv', index = None)

In [2]:
s = pd.read_csv('sample_submission-1646898326432-1664166963268.csv')

In [3]:
s.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20848800 entries, 0 to 20848799
Data columns (total 2 columns):
 #   Column  Dtype
---  ------  -----
 0   id      int64
 1   yield   int64
dtypes: int64(2)
memory usage: 318.1 MB


In [6]:
ss.columns

Index(['0', '1'], dtype='object')

In [9]:
new = {'0': 'id', '1': 'yield'}
ss = ss.rename(columns=new)

In [4]:
ss = pd.read_csv('submission1-Copy1.csv')

In [8]:
ss['0'] = ss['0'].astype('int64')

In [10]:
ss['yield'] = ss['yield'].astype('int64')

In [12]:
ss.to_csv('submit1.csv', index = None)

In [11]:
ss.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20848800 entries, 0 to 20848799
Data columns (total 2 columns):
 #   Column  Dtype
---  ------  -----
 0   id      int64
 1   yield   int64
dtypes: int64(2)
memory usage: 318.1 MB


In [106]:
s = pd.read_csv('submission_v1.csv')

In [107]:
s

,0,1
0,0,-0.013816
1,1,-0.013816
2,2,-0.013816
3,3,-0.013816
4,4,-0.013816
...,...,...
1048570,1048570,-0.013236
1048571,1048571,-0.012540
1048572,1048572,-0.013503
1048573,1048573,0.003890
